# LangSmith and Evaluation Overview with AI Makerspace

Today we'll be looking at an amazing tool:

[LangSmith](https://docs.smith.langchain.com/)!

This tool will help us monitor, test, debug, and evaluate our LangChain applications - and more!

We'll also be looking at a few Advanced Retrieval techniques along the way - and evaluate it using LangSmith!

✋BREAKOUT ROOM #2:
- Task 1: Dependencies and OpenAI API Key
- Task 2: LCEL RAG Chain
- Task 3: Setting Up LangSmith
- Task 4: Examining the Trace in LangSmith!
- Task 5: Create Testing Dataset
- Task 6: Evaluation

## Task 1: Dependencies and OpenAI API Key

We'll be using OpenAI's suite of models today to help us generate and embed our documents for a simple RAG system built on top of LangChain's blogs!

> NOTE: DO NOT RUN THE FOLLOWING CELL IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install langchain_core langchain_openai langchain_community langchain-qdrant qdrant-client langsmith openai tiktoken cohere lxml -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.9/258.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

#### Asyncio Bug Handling

This is necessary for Colab.

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task #2: Create a Simple RAG Application Using Qdrant, Hugging Face, and LCEL

Now that we have a grasp on how LCEL works, and how we can use LangChain and Hugging Face to interact with our data - let's step it up a notch and incorporate Qdrant!

## LangChain Powered RAG

First and foremost, LangChain provides a convenient way to store our chunks and their embeddings.

It's called a `VectorStore`!

We'll be using QDrant as our `VectorStore` today. You can read more about it [here](https://qdrant.tech/documentation/).

Think of a `VectorStore` as a smart way to house your chunks and their associated embedding vectors. The implementation of the `VectorStore` also allows for smarter and more efficient search of our embedding vectors - as the method we used above would not scale well as we got into the millions of chunks.

Otherwise, the process remains relatively similar under the hood!

We'll use a SiteMapLoader to scrape the LangChain blogs - which will serve as our data for today!

### Data Collection

We'll be leveraging the `SitemapLoader` to load our PDF directly from the web!

In [3]:
from langchain.document_loaders import SitemapLoader

documents = SitemapLoader(web_path="https://blog.langchain.dev/sitemap-posts.xml").load()

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 283/283 [00:08<00:00, 32.31it/s]


### Chunking Our Documents

Let's do the same process as we did before with our `RecursiveCharacterTextSplitter` - but this time we'll use ~200 tokens as our max chunk size!

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
    length_function = len,
)

split_chunks = text_splitter.split_documents(documents)

In [5]:
len(split_chunks)

6029

Alright, now we have 516 ~200 token long documents.

Let's verify the process worked as intended by checking our max document length.

In [6]:
max_chunk_length = 0

for chunk in split_chunks:
  max_chunk_length = max(max_chunk_length, len(chunk.page_content))

print(max_chunk_length)

499


Perfect! Now we can carry on to creating and storing our embeddings.

### Embeddings and Vector Storage

We'll use the `text-embedding-3-small` embedding model again - and `Qdrant` to store all our embedding vectors for easy retrieval later!

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

qdrant_vectorstore = Qdrant.from_documents(
    documents=split_chunks,
    embedding=embedding_model,
    location=":memory:"
)

Now let's set up our retriever, just as we saw before, but this time using LangChain's simple `as_retriever()` method!

In [8]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

#### Back to the Flow

We're ready to move to the next step!

### Setting up our RAG

We'll use the LCEL we touched on earlier to create a RAG chain.

Let's think through each part:

1. First we need to retrieve context
2. We need to pipe that context to our model
3. We need to parse that output

Let's start by setting up our prompt again, just so it's fresh in our minds!

####🏗️ Activity #2:

Complete the prompt so that your RAG application answers queries based on the context provided, but *does not* answer queries if the context is unrelated to the query.

In [9]:
from langchain.prompts import ChatPromptTemplate

base_rag_prompt_template = """\
You are a helpful assistant that answers questions based on the context provided.
You have access to the following information:

Context:
{context}

Question:
{question}

If context is unrelated to question, say "I don't know".
"""

base_rag_prompt = ChatPromptTemplate.from_template(base_rag_prompt_template)

We'll set our Generator - `gpt-4o` in this case - below!

In [10]:
from langchain_openai.chat_models import ChatOpenAI

base_llm = ChatOpenAI(model="gpt-4o-mini", tags=["base_llm"])

#### Our RAG Chain

Notice how we have a bit of a more complex chain this time - that's because we want to return our sources with the response.

Let's break down the chain step-by-step:

1. We invoke the chain with the `question` item. Notice how we only need to provide `question` since both the retreiver and the `"question"` object depend on it.
  - We also chain our `"question"` into our `retriever`! This is what ultimately collects the context through Qdrant.
2. We assign our collected context to a `RunnablePassthrough()` from the previous object. This is going to let us simply pass it through to the next step, but still allow us to run that section of the chain.
3. We finally collect our response by chaining our prompt, which expects both a `"question"` and `"context"`, into our `llm`. We also, collect the `"context"` again so we can output it in the final response object.

The key thing to keep in mind here is that we need to pass our context through *after* we've retrieved it - to populate the object in a way that doesn't require us to call it or try and use it for something else.

In [11]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": base_rag_prompt | base_llm, "context": itemgetter("context")}
)

Let's get a visual understanding of our chain!

> NOTE: DO NOT RUN THIS CELL IF YOU'RE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
# !pip install -qU grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.2 MB/s eta 0:00:00


In [12]:
print(retrieval_augmented_qa_chain.get_graph().draw_ascii())

          +---------------------------------+      
          | Parallel<context,question>Input |      
          +---------------------------------+      
                    **            **               
                  **                **             
                **                    **           
         +--------+                     **         
         | Lambda |                      *         
         +--------+                      *         
              *                          *         
              *                          *         
              *                          *         
  +----------------------+          +--------+     
  | VectorStoreRetriever |          | Lambda |     
  +----------------------+          +--------+     
                    **            **               
                      **        **                 
                        **    **                   
          +----------------------------------+     
          | 

Let's try another visual representation:

![image](https://i.imgur.com/Ad31AhL.png)

Let's test our chain out!

In [13]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What's new in LangChain v0.2?"})

In [14]:
response["response"].content

'LangChain v0.2 includes several improvements and new features based on community feedback from the previous version. Key updates are:\n\n1. Full separation of the langchain package from langchain-community.\n2. New and versioned documentation.\n3. A more mature and controllable agent framework.\n4. Improved standardization of the LLM interface, particularly regarding tool calling.\n5. Streaming support.\n6. Over 30 new partner packages.\n\nThis version aims to enhance stability and security, with the full release expected shortly after the pre-release announcement.'

In [15]:
for context in response["context"]:
  print("Context:")
  print(context)
  print("----")

Context:
page_content='Four months ago, we released the first stable version of LangChain. Today, we are following up by announcing a pre-release of langchain v0.2.This release builds upon the foundation laid in v0.1 and incorporates community feedback. We’re excited to share that v0.2 brings: A much-desired full separation of langchain and langchain-community New (and versioned!) docs A more mature and controllable agent framework Improved LLM interface standardization, particularly around tool callingBetter' metadata={'source': 'https://blog.langchain.dev/langchain-v02-leap-to-stability/', 'loc': 'https://blog.langchain.dev/langchain-v02-leap-to-stability/', 'lastmod': '2024-05-16T22:26:07.000Z', '_id': '8e7435ff67f14ce7a048997cd573f6fb', '_collection_name': '4857d09486c4435abb838a2cb8563272'}
----
Context:
page_content='LangChain v0.2: A Leap Towards Stability




















































Skip to content
















Case Studies




In the Loop




LangCha

Let's see if it can handle a query that is totally unrelated to the source documents.

In [16]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the airspeed velocity of an unladen swallow?"})

In [17]:
response["response"].content

"I don't know."

## Task 3: Setting Up LangSmith

Now that we have a chain - we're ready to get started with LangSmith!

We're going to go ahead and use the following `env` variables to get our Colab notebook set up to start reporting.

If all you needed was simple monitoring - this is all you would need to do!

In [18]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"LangSmith - {unique_id}"

### LangSmith API

In order to use LangSmith - you will need a beta key, you can join the queue through the `Beta Sign Up` button on LangSmith's homepage!

Join [here](https://www.langchain.com/langsmith)

In [19]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Let's test our our first generation!

In [21]:
retrieval_augmented_qa_chain.invoke({"question" : "What is LangSmith?"}, {"tags" : ["Demo Run"]})['response']

AIMessage(content='LangSmith is a framework built on LangChain, designed to track and enhance the development and performance of AI-powered products, particularly those utilizing large language models (LLMs). It offers features like observability in the development lifecycle, and its SDK allows for fine-grain control and customizability. LangSmith facilitates improved user experience and development efficiency by providing tools for tracking function performance and managing complex configurations in AI applications.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 919, 'total_tokens': 1003, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_64e0ac9789', 'finish_reason': 'stop', 'logprobs': None}, id='r

## Task 4: Examining the Trace in LangSmith!

Head on over to your LangSmith web UI to check out how the trace looks in LangSmith!

#### 🏗️ Activity #1:

Include a screenshot of your trace and explain what it means.

## Task 5: Loading Our Testing Set

In [22]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 119 (delta 36), reused 40 (delta 10), pack-reused 8 (from 1)
Receiving objects: 100% (119/119), 78.04 MiB | 30.36 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [23]:
import pandas as pd

test_df = pd.read_csv("DataRepository/langchain_blog_test_data.csv")

In [ ]:
test_df

Now we can set up our LangSmith client - and we'll add the above created dataset to our LangSmith instance!

> NOTE: Read more about this process [here](https://docs.smith.langchain.com/old/evaluation/faq/manage-datasets#create-from-list-of-values)

In [24]:
from langsmith import Client

client = Client()

dataset_name = "langsmith-demo-dataset-aie6-triples-v3"

dataset = client.create_dataset(
    dataset_name=dataset_name, description="LangChain Blog Test Questions"
)

for triplet in test_df.iterrows():
  triplet = triplet[1]
  client.create_example(
      inputs={"question" : triplet["question"], "context": triplet["context"]},
      outputs={"answer" : triplet["answer"]},
      dataset_id=dataset.id
  )

## Task 6: Evaluation

Now we can run the evaluation!

We'll need to start by preparing some custom data preparation functions to ensure our chain works with the expected inputs/outputs from the `evaluate` process in LangSmith.

> NOTE: More reading on this available [here](https://docs.smith.langchain.com/how_to_guides/evaluation/evaluate_llm_application#evaluate-a-langchain-runnable)

In [25]:
def prepare_data_ref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "reference" : example.outputs["answer"],
      "input" : example.inputs["question"]
  }

def prepare_data_noref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "input" : example.inputs["question"]
  }

def prepare_context_ref(run, example):
  return {
      "prediction" : run.outputs["response"],
      "reference" : example.inputs["context"],
      "input" : example.inputs["question"]
  }

We'll be using a few custom evaluators to evaluate our pipeline, as well as a few "built in" methods!

Check out the built-ins [here](https://docs.smith.langchain.com/reference/sdk_reference/langchain_evaluators)!

In [26]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

eval_llm = ChatOpenAI(model="gpt-4o-mini", tags=["eval_llm"])

cot_qa_evaluator = LangChainStringEvaluator("cot_qa",  config={"llm":eval_llm}, prepare_data=prepare_context_ref)

unlabeled_dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria" : {
            "dopeness" : "Is the answer to the question dope, meaning cool - awesome - and legit?"
        },
        "llm" : eval_llm,
    },
    prepare_data=prepare_data_noref
)

labeled_score_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "criteria": {
            "accuracy": "Is the generated answer the same as the reference answer?"
        },
    },
    prepare_data=prepare_data_ref
)

base_rag_results = evaluate(
    retrieval_augmented_qa_chain.invoke,
    data=dataset_name,
    evaluators=[
        cot_qa_evaluator,
        unlabeled_dopeness_evaluator,
        labeled_score_evaluator,
        ],
    experiment_prefix="Base RAG Evaluation"
)

View the evaluation results for experiment: 'Base RAG Evaluation-024c77d7' at:
https://smith.langchain.com/o/e106fdae-1163-4ad0-b46b-09a4850df972/datasets/70e5e8be-2211-46d9-8be2-baccb949cfcf/compare?selectedSessions=e0f305ee-17be-497a-a70a-6d05bbfe8816




0it [00:00, ?it/s]

#### ❓Question #1:

What conclusions can you draw about the above results?

Describe in your own words what the metrics are expressing.
# Evaluation Data Analysis Report

## **Full Dataset**
| status   | error | latency       | tokens | total_cost | cot contextual accuracy | score_string:accuracy | dopeness |
|----------|-------|---------------|--------|------------|-------------------------|-----------------------|----------|
| success  |       | 1.565510035   | 883     | 0.00014775 | CORRECT                 | 10                    | N        |
| success  |       | 1.169227123   | 895     | 0.00015    | CORRECT                 | 2                     | Y        |
| success  |       | 2.827636003   | 1083    | 0.00023445 | CORRECT                 | 9                     | N        |
| success  |       | 2.055365086   | 1009    | 0.00019545 | CORRECT                 | 9                     | N        |
| success  |       | 3.537876129   | 1142    | 0.00027075 | CORRECT                 | 3                     | N        |
| success  |       | 2.816374063   | 1032    | 0.00022095 | INCORRECT               | 2                     | Y        |
| success  |       | 1.204406977   | 959     | 0.0001677  | INCORRECT               | 10                    | N        |
| success  |       | 2.377774      | 990     | 0.00020655 | INCORRECT               | 2                     | Y        |
| success  |       | 1.169979811   | 896     | 0.0001479  | CORRECT                 | 10                    | Y        |
| success  |       | 1.846786976   | 1049    | 0.00018345 | CORRECT                 | 7                     | N        |
| success  |       | 0.942778826   | 928     | 0.0001518  | CORRECT                 | 10                    | N        |
| success  |       | 1.151479006   | 944     | 0.00015105 | CORRECT                 | 9                     | N        |
| success  |       | 1.78966403    | 1112    | 0.00019695 | CORRECT                 | 5                     | Y        |
| success  |       | 2.141712904   | 995     | 0.00020145 | CORRECT                 | 9                     | Y        |
| success  |       | 1.464936972   | 1131    | 0.00018585 | CORRECT                 | 7                     | N        |
| success  |       | 0.869203091   | 1107    | 0.00017415 | CORRECT                 | 8                     | N        |
| success  |       | 1.899935961   | 1068    | 0.0001935  | CORRECT                 | 10                    | N        |
| success  |       | 4.281825066   | 895     | 0.00014775 | CORRECT                 | 10                    | N        |
| success  |       | 2.038258791   | 958     | 0.0001788  | INCORRECT               | 10                    | Y        |
| success  |       | 2.496733189   | 1000    | 0.0002112  | CORRECT                 | 7                     | Y        |
| success  |       | 1.464993954   | 875     | 0.00015645 | CORRECT                 | 8                     | N        |
| success  |       | 5.67979598    | 1158    | 0.00030195 | CORRECT                 | 10                    | Y        |
| success  |       | 2.074322939   | 1129    | 0.00020175 | CORRECT                 | 8                     | Y        |

---

## **Key Metrics**
### 1. **General Statistics**
- **Total Entries**: 23
- **Success Rate**: 100% (all entries have `status: success`).
- **Errors**: None (all `error` fields are empty).

### 2. **Latency**
- **Average Latency**: **2.12 seconds**
- **Range**: 0.869s (min) to 5.680s (max).
- **Outliers**: 
  - Highest latency: **5.68s** (tokens: 1158, total_cost: 0.00030195).
  - Lowest latency: **0.869s** (tokens: 1107, total_cost: 0.00017415).

### 3. **Tokens & Cost**
- **Token Range**: 875 to 1158.
- **Total Cost Range**: $0.00014775 to $0.00030195.
- **Cost per Token**: ~$0.000000167 (consistent across entries).

### 4. **Accuracy**
- **CORRECT**: 19 entries (**82.6%**).
- **INCORRECT**: 4 entries (**17.4%**).
- **Score Distribution**:
  - **High Scores (8-10)**: 13 entries (68.4% of CORRECT responses).
  - **Mid Scores (5-7)**: 4 entries (21.1% of CORRECT responses).
  - **Low Scores (2-3)**: 2 entries (10.5% of CORRECT responses).

### 5. **Dopeness (Y/N)**
- **Y (Yes)**: 10 entries (**43.5%**).
- **N (No)**: 13 entries (**56.5%**).
- **Dopeness vs. Accuracy**:
  - `Y`: 7 CORRECT, 3 INCORRECT (**70% accuracy for Y**).
  - `N`: 12 CORRECT, 1 INCORRECT (**92.3% accuracy for N**).

---

## **Detailed Analysis**
### 1. **Latency vs. Tokens**
- **Weak Correlation**: Higher tokens (e.g., 1158) sometimes correlate with higher latency (5.68s), but exceptions exist (e.g., 895 tokens with 4.28s latency).
- **Efficiency**: Lower token counts (e.g., 875 tokens) often have lower latency (~1.46s).

### 2. **Cost Efficiency**
- **Most Cost-Effective Entry**:  
  - Tokens: 875, Cost: $0.00015645, Latency: 1.46s (row 21).
- **Least Cost-Effective Entry**:  
  - Tokens: 1158, Cost: $0.00030195, Latency: 5.68s (row 22).

### 3. **Accuracy Anomalies**
- **Unexpected Scores**:  
  - `CORRECT` with `score: 2` (row 2).  
  - `INCORRECT` with `score: 10` (rows 7 and 19).  
  - Suggests potential misalignment between `score` and `cot contextual accuracy`.

### 4. **Dopeness Insights**
- **Y (Yes) Entries**:  
  - Higher error rate (**30% incorrect**) compared to N entries (**7.7% incorrect**).  
  - Scores range widely (2-10), indicating variability in task complexity or quality.  
- **N (No) Entries**:  
  - More reliable (**92.3% accuracy**) and consistent scores (7-10).  

---

## **Critical Observations**
1. **Dopeness Impact**: Tasks marked `Y` are riskier (3/10 incorrect) but may involve more complex or experimental workflows.  
2. **Score-Accuracy Mismatch**: High scores (e.g., 10) occasionally accompany `INCORRECT` results (rows 7, 19), suggesting scoring criteria may need refinement.  
3. **Latency Outliers**: The entry with 5.68s latency (row 22) is a performance bottleneck despite high accuracy.  
4. **Cost Scaling**: Total cost increases linearly with token count, but cost per token remains stable.  

---

Based on the evaluation results from our Base RAG system, we can draw several important conclusions:

1. **Quality of Retrieved Context**: The COT QA evaluator helps us understand if our retrieval system is bringing back relevant context that supports answering user questions accurately. This metric indicates how well our vector search and embedding model are performing.

2. **Answer Accuracy**: The labeled score evaluator measures how closely our generated answers match reference answers. This shows whether our RAG system is correctly interpreting the retrieved context and formulating appropriate responses.

3. **User Experience**: The "dopeness" evaluator provides insight into how engaging or appealing our responses are beyond mere factual accuracy. This is important for user satisfaction and engagement.

4. **Error Analysis**: By examining cases where scores are low, we can identify patterns of failure - whether they're due to poor retrieval, incorrect interpretation of context, or issues with the LLM's response generation.

5. **Comparative Performance**: These metrics establish a baseline that allows us to measure improvements as we refine our RAG implementation, adjust prompts, or experiment with different models or retrieval techniques.

The metrics collectively express both the technical performance (accuracy, relevance) and user-oriented qualities of our system, providing a comprehensive view of its effectiveness in real-world applications.